In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 設定 Matplotlib 顯示中文字體 (請根據您的作業系統選擇字體)
# For Windows
# plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
# For Mac
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False # 解決負號顯示問題


# --- 載入資料 ---
# 提示：如果檔案路徑或名稱有中文，建議在前面加上 u
file_path = u'2023資料剖析後內容1~8月芝山到天母0609整理二版.csv' 
try:
    # 嘗試用 utf-8-sig 來讀取，它可以處理帶有 BOM 的 UTF-8 檔案
    df = pd.read_csv(file_path, encoding='utf-8-sig')
except UnicodeDecodeError:
    # 如果失敗，嘗試用 big5 編碼
    df = pd.read_csv(file_path, encoding='big5')


# --- 初步探索 ---
print("資料前五筆：")
print(df.head())
print("\n資料基本資訊：")
df.info()

# 觀察 'Stp1' 和 'Time2' 的唯一值
print("\n起點站 (Stp1) 的唯一值：")
print(df['Stp1'].unique())

print("\n騎乘時間 (Time2) 的一些範例值：")
print(df['Time2'].unique()[:10])


資料前五筆：
       Date  Week     Time1 AM/PM           Stp1          Stp2  Time2  \
0  20230101     7  04:00:00    AM  捷運芝山站(2號出口)_1  臺北市立大學(天母校區)  16:07   
1  20230101     7  05:00:00    AM    捷運芝山站(2號出口)  臺北市立大學(天母校區)  21:22   
2  20230101     7  08:00:00    AM  捷運芝山站(2號出口)_1  臺北市立大學(天母校區)  09:39   
3  20230101     7  12:00:00    PM  捷運芝山站(2號出口)_1  臺北市立大學(天母校區)  12:06   
4  20230101     7  12:00:00    PM  捷運芝山站(2號出口)_1  臺北市立大學(天母校區)  10:59   

   Unnamed: 7  
0         NaN  
1         NaN  
2         NaN  
3         NaN  
4         NaN  

資料基本資訊：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5016 entries, 0 to 5015
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Date        5016 non-null   int64  
 1   Week        5016 non-null   int64  
 2   Time1       5016 non-null   object 
 3   AM/PM       5016 non-null   object 
 4   Stp1        5016 non-null   object 
 5   Stp2        5016 non-null   object 
 6   Time2     

In [3]:
# 移除最後一行的文字說明
df = df.iloc[:-1].copy()

def parse_time_to_seconds(time_str):
    """將 MM:SS 或 HH:MM:SS 格式的字串轉為總秒數，錯誤格式返回 None"""
    try:
        # pd.to_timedelta 可以處理多種時間格式
        # 我們強制單位為秒，並取其總秒數
        return pd.to_timedelta(time_str).total_seconds()
    except (ValueError, TypeError):
        # 如果格式錯誤或值為空，返回 None
        return None

# 應用函數，並建立新欄位 'TotalSeconds'
df['TotalSeconds'] = df['Time2'].apply(parse_time_to_seconds)

# --- 清理轉換後的資料 ---
# 移除轉換失敗的資料列 (TotalSeconds 是 NaT 或 None)
original_rows = len(df)
df.dropna(subset=['TotalSeconds'], inplace=True)
print(f"移除了 {original_rows - len(df)} 筆無法解析時間的資料。")

# 將秒數轉為整數
df['TotalSeconds'] = df['TotalSeconds'].astype(int)

# --- 處理極端值 ---
# 騎乘時間少於 3 分鐘 (180秒) 或多於 60 分鐘 (3600秒) 可能有問題，先移除
original_rows = len(df)
df = df[(df['TotalSeconds'] >= 180) & (df['TotalSeconds'] <= 3600)]
print(f"移除了 {original_rows - len(df)} 筆騎乘時間極端的資料。")

print("\n清理後的時間資料分佈：")
print(df['TotalSeconds'].describe())

移除了 5014 筆無法解析時間的資料。
移除了 1 筆騎乘時間極端的資料。

清理後的時間資料分佈：
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: TotalSeconds, dtype: float64


In [4]:
# 將 Date 轉為字串格式
df['Date'] = df['Date'].astype(str)

# 合併 Date 和 Time1 欄位
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time1'], format='%Y%m%d %H:%M:%S')

print("\n轉換後的日期時間欄位：")
print(df[['Date', 'Time1', 'Datetime']].head())


轉換後的日期時間欄位：
Empty DataFrame
Columns: [Date, Time1, Datetime]
Index: []


In [11]:
def normalize_exit(stp_name):
    if '1號' in str(stp_name):
        return 'Exit_1'
    else:
        return 'Exit_2'

df['StartExit'] = df['Stp1'].apply(normalize_exit)

print("\n標準化後的出口欄位：")
print(df['StartExit'].value_counts())


標準化後的出口欄位：
Series([], Name: count, dtype: int64)


In [17]:
print("目前的 df 欄位列表：")
print(df.columns)

# 檢查 'StartExit' 是否存在
if 'StartExit' in df.columns:
    print("\n[成功] 'StartExit' 欄位存在於 DataFrame 中。")
else:
    print("\n[錯誤] 'StartExit' 欄位不存在！請重啟核心並從頭執行。")

目前的 df 欄位列表：
Index(['Date', 'Week', 'Time1', 'AM/PM', 'Stp1', 'Stp2', 'Time2', 'Unnamed: 7',
       'TotalSeconds', 'Datetime', 'Hour', 'DayOfWeek', 'Month', 'IsWeekend',
       'IsRushHour'],
      dtype='object')

[錯誤] 'StartExit' 欄位不存在！請重啟核心並從頭執行。


In [19]:
# --- Step 3: 特徵工程 (快速修復版) ---

# ... (前面提取 Hour, DayOfWeek 等特徵的程式碼保持不變) ...
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek
df['Month'] = df['Datetime'].dt.month
df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)
df['IsRushHour'] = df['Hour'].apply(lambda h: 1 if (7 <= h <= 9) or (17 <= h <= 19) else 0)


# --- 將 StartExit 進行 One-Hot Encoding ---
df = pd.get_dummies(df, columns=['StartExit'], prefix='Exit', dtype=int)


# --- 動態獲取特徵欄位 ---
# 先定義好基礎特徵
base_features = ['Hour', 'DayOfWeek', 'Month', 'IsWeekend', 'IsRushHour']
# 找出所有以 'Exit_' 開頭的欄位 (這是 get_dummies 產生的)
exit_features = [col for col in df.columns if col.startswith('Exit_')]

# 將基礎特徵和出口特徵合併成最終的特徵列表
features = base_features + exit_features

# --- 檢視我們建立好的特徵 ---
print("\n最終用於建模的特徵欄位預覽：")
features_to_show = ['TotalSeconds'] + features # 將目標也加進來顯示
print(df[features_to_show].head())


# --- 更新後續訓練時使用的特徵列表 ---
# 在 Step 4 中，你就不需要再手動定義 features 列表了，直接使用這裡產生的即可
print("\n用於訓練的特徵:", features)


KeyError: 'Datetime'

In [20]:
from sklearn.preprocessing import OneHotEncoder

# --- Step 3: 特徵工程 (修正版) ---

# 從 'Datetime' 欄位中提取更多特徵
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek  # 星期一=0, 星期日=6
df['Month'] = df['Datetime'].dt.month
df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

# 建立尖峰時段特徵
def is_rush_hour(hour):
    if (7 <= hour <= 9) or (17 <= hour <= 19):
        return 1
    return 0
df['IsRushHour'] = df['Hour'].apply(is_rush_hour)

# --- 使用 OneHotEncoder 處理 'StartExit' ---
# 1. 實例化 OneHotEncoder
#    handle_unknown='ignore' 可以在未來預測時忽略未見過的類別
#    sparse_output=False 讓它直接輸出 numpy array 而不是稀疏矩陣
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# 2. Fit and transform the 'StartExit' column
#    需要將 df[['StartExit']] 轉為 2D array
encoded_features = ohe.fit_transform(df[['StartExit']])

# 3. 獲取新欄位的名稱 (例如 'StartExit_Exit_1', 'StartExit_Exit_2')
new_feature_names = ohe.get_feature_names_out(['StartExit'])

# 4. 將編碼後的特徵轉換為 DataFrame
encoded_df = pd.DataFrame(encoded_features, columns=new_feature_names, index=df.index)

# 5. 將新的編碼欄位合併回原始 DataFrame，並刪除舊的 'StartExit'
df = pd.concat([df, encoded_df], axis=1)
df.drop('StartExit', axis=1, inplace=True)


# --- 檢視我們建立好的特徵 ---
# 由於欄位名稱改變了，我們也更新一下要顯示的列表
print("\n最終用於建模的特徵欄位預覽：")
features_to_show = ['TotalSeconds', 'Hour', 'DayOfWeek', 'IsWeekend', 'IsRushHour'] + list(new_feature_names)
print(df[features_to_show].head())

# --- 更新後續訓練時使用的特徵列表 ---
# 在 Step 4 中，features 列表也需要更新
features = ['Hour', 'DayOfWeek', 'Month', 'IsWeekend', 'IsRushHour'] + list(new_feature_names)
print("\n用於訓練的特徵:", features)
Use code with caution.
Python
# 優點： 這是機器學習專案的標準做法，更為嚴謹且可靠。
# 方案二：(快速修復) 動態產生特徵列表
# 如果您想繼續使用 pd.get_dummies，不想改動太多，那麼我們需要讓程式碼變得更「動態」，不要寫死欄位名稱。
# 請將您原本 Step 3 的程式碼替換成以下內容：
# --- Step 3: 特徵工程 (快速修復版) ---

# ... (前面提取 Hour, DayOfWeek 等特徵的程式碼保持不變) ...
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek
df['Month'] = df['Datetime'].dt.month
df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)
df['IsRushHour'] = df['Hour'].apply(lambda h: 1 if (7 <= h <= 9) or (17 <= h <= 19) else 0)


# --- 將 StartExit 進行 One-Hot Encoding ---
df = pd.get_dummies(df, columns=['StartExit'], prefix='Exit', dtype=int)


# --- 動態獲取特徵欄位 ---
# 先定義好基礎特徵
base_features = ['Hour', 'DayOfWeek', 'Month', 'IsWeekend', 'IsRushHour']
# 找出所有以 'Exit_' 開頭的欄位 (這是 get_dummies 產生的)
exit_features = [col for col in df.columns if col.startswith('Exit_')]

# 將基礎特徵和出口特徵合併成最終的特徵列表
features = base_features + exit_features

# --- 檢視我們建立好的特徵 ---
print("\n最終用於建模的特徵欄位預覽：")
features_to_show = ['TotalSeconds'] + features # 將目標也加進來顯示
print(df[features_to_show].head())


# --- 更新後續訓練時使用的特徵列表 ---
# 在 Step 4 中，你就不需要再手動定義 features 列表了，直接使用這裡產生的即可
print("\n用於訓練的特徵:", features)

SyntaxError: invalid syntax (910402959.py, line 49)

In [21]:
# 從 'Datetime' 欄位中提取更多特徵
df['Hour'] = df['Datetime'].dt.hour
df['DayOfWeek'] = df['Datetime'].dt.dayofweek  # 星期一=0, 星期日=6
df['Month'] = df['Datetime'].dt.month
df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0) # 1 代表週末

# 建立尖峰時段特徵 (早上7-9點, 下午5-7點)
def is_rush_hour(hour):
    if (7 <= hour <= 9) or (17 <= hour <= 19):
        return 1
    return 0
df['IsRushHour'] = df['Hour'].apply(is_rush_hour)

# 將 StartExit 進行 One-Hot Encoding
# 這會將類別特徵轉換為模型可以理解的 0/1 數值格式
df = pd.get_dummies(df, columns=['StartExit'], prefix='Exit')


# 檢視我們建立好的特徵
print("\n最終用於建模的特徵欄位預覽：")
features_to_show = ['TotalSeconds', 'Hour', 'DayOfWeek', 'IsWeekend', 'IsRushHour', 'Exit_Exit_1', 'Exit_Exit_2']
print(df[features_to_show].head())

KeyError: 'Datetime'

In [23]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

# --- 準備訓練資料 ---
# X 是我們的特徵，y 是我們的目標
features = ['Hour', 'DayOfWeek', 'Month', 'IsWeekend', 'IsRushHour', 'Exit_Exit_1', 'Exit_Exit_2']
target = 'TotalSeconds'

X = df[features]
y = df[target]

# --- 分割訓練集與測試集 ---
# 80% 用於訓練，20% 用於測試模型的表現
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"訓練集大小: {X_train.shape[0]} 筆")
print(f"測試集大小: {X_test.shape[0]} 筆")


# --- 初始化並訓練模型 ---
# LGBMRegressor 是一個用於迴歸問題的模型
lgbm = lgb.LGBMRegressor(random_state=42)

print("\n開始訓練模型...")
lgbm.fit(X_train, y_train)
print("模型訓練完成！")

KeyError: "None of [Index(['Hour', 'DayOfWeek', 'Month', 'IsWeekend', 'IsRushHour', 'Exit_Exit_1',\n       'Exit_Exit_2'],\n      dtype='object')] are in the [columns]"

In [24]:
from sklearn.metrics import mean_absolute_error, r2_score

# 在測試集上進行預測
y_pred = lgbm.predict(X_test)

# 計算評估指標
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\n--- 模型評估結果 ---")
print(f"平均絕對誤差 (MAE): {mae:.2f} 秒")
print(f"這代表我們的預測平均會與實際騎乘時間相差約 {mae/60:.2f} 分鐘。")
print(f"R-squared (R²): {r2:.2f}")
print("R² 衡量模型對資料變異的解釋能力，越接近1越好。")

# 視覺化預測結果 vs 實際結果
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--', color='red', linewidth=2)
plt.title('實際騎乘時間 vs. 模型預測時間')
plt.xlabel('實際時間 (秒)')
plt.ylabel('預測時間 (秒)')
plt.grid(True)
plt.show()

NameError: name 'lgbm' is not defined

In [25]:
# 獲取特徵重要性
feature_imp = pd.DataFrame(sorted(zip(lgbm.feature_importances_, X.columns)), columns=['Value','Feature'])

plt.figure(figsize


SyntaxError: incomplete input (2968803707.py, line 4)